In [300]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [301]:
# Initialize df for team data
teams_df = pd.DataFrame(columns=['Season','Team', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS'])
# Populate Team Data with web scraping
def get_team_data():
    season = []
    teams = []
    W = []
    L = []
    T = []
    W_L = []
    PF = []
    PA = []
    PD = []
    MoV = []
    SoS = []
    SRS = []
    OSRS = []
    DSRS = []
    
    for year in range(2019, 2023):
        html_text = requests.get(f'https://www.pro-football-reference.com/years/{year}/index.htm').text
        soup = BeautifulSoup(html_text, 'lxml')
        tables = soup.find_all('tbody')
        
        for table in tables:
            teams_data = table.find_all('tr', class_="")
            for team_data in teams_data:
                season.append(year)
                teams.append(team_data.find('a').text)
                W.append(team_data.find_all('td')[0].text)
                L.append(team_data.find_all('td')[1].text)
                T.append(team_data.find_all('td')[2].text)
                W_L.append(team_data.find_all('td')[3].text)
                PF.append(team_data.find_all('td')[4].text)
                PA.append(team_data.find_all('td')[5].text)
                PD.append(team_data.find_all('td')[6].text)
                MoV.append(team_data.find_all('td')[7].text)
                SoS.append(team_data.find_all('td')[8].text)
                SRS.append(team_data.find_all('td')[9].text)
                OSRS.append(team_data.find_all('td')[10].text)
                DSRS.append(team_data.find_all('td')[11].text)

    teams_df['Team'] = teams
    teams_df['W'] = W
    teams_df['L'] = L
    teams_df['T'] = T
    teams_df['W-L%'] = W_L
    teams_df['PF'] = PF
    teams_df['PA'] = PA
    teams_df['PD'] = PD
    teams_df['MoV'] = MoV
    teams_df['SoS'] = SoS
    teams_df['SRS'] = SRS
    teams_df['OSRS'] = OSRS
    teams_df['DSRS'] = DSRS
    teams_df['Season'] = season
get_team_data()

In [302]:
# Display teams data
teams_df

,Season,Team,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
0,2019,New England Patriots,12,4,0,.750,420,225,195,12.2,-1.8,10.4,2.8,7.6
1,2019,Buffalo Bills,10,6,0,.625,314,259,55,3.4,-1.3,2.2,-3.5,5.7
2,2019,New York Jets,7,9,0,.438,276,359,-83,-5.2,-1.1,-6.3,-5.7,-0.6
3,2019,Miami Dolphins,5,11,0,.313,306,494,-188,-11.8,0.2,-11.6,-2.4,-9.1
4,2019,Baltimore Ravens,14,2,0,.875,531,282,249,15.6,0.1,15.6,11.0,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2022,Atlanta Falcons,7,10,0,.412,365,386,-21,-1.2,-0.9,-2.1,-0.1,-2.0
124,2022,San Francisco 49ers,13,4,0,.765,450,277,173,10.2,-2.3,7.9,3.3,4.6
125,2022,Seattle Seahawks,9,8,0,.529,407,401,6,0.4,-0.8,-0.5,1.9,-2.4
126,2022,Los Angeles Rams,5,12,0,.294,307,384,-77,-4.5,0.5,-4.0,-4.1,0.0


In [303]:
# Sort teams alphabetically for easy merging
teams_df_sort = teams_df.sort_values(by = 'Team').reset_index(drop=True)

In [304]:
teams_df_sort

,Season,Team,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
0,2022,Arizona Cardinals,4,13,0,.235,340,449,-109,-6.4,0.2,-6.2,-1.9,-4.3
1,2019,Arizona Cardinals,5,10,1,.344,361,442,-81,-5.1,1.8,-3.2,-0.3,-2.9
2,2021,Arizona Cardinals,11,6,0,.647,449,366,83,4.9,0.3,5.2,3.6,1.6
3,2020,Arizona Cardinals,8,8,0,.500,410,367,43,2.7,-0.1,2.6,1.5,1.0
4,2022,Atlanta Falcons,7,10,0,.412,365,386,-21,-1.2,-0.9,-2.1,-0.1,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2022,Tennessee Titans,7,10,0,.412,298,359,-61,-3.6,0.0,-3.6,-4.7,1.1
124,2022,Washington Commanders,8,8,1,.500,321,343,-22,-1.3,0.4,-0.9,-3.5,2.6
125,2020,Washington Football Team,7,9,0,.438,335,329,6,0.4,-1.2,-0.8,-4.1,3.2
126,2021,Washington Football Team,7,10,0,.412,335,434,-99,-5.8,1.6,-4.2,-2.7,-1.5


In [305]:
# # Initialize df for offense redzone data
# off_red_df = pd.DataFrame(columns=['Team', 'off_red_per'])
# # Populate offense redzone data with web scraping
# def get_off_red_data():
#     teams = []
#     percent = []
#     html_text = requests.get('https://www.teamrankings.com/nfl/stat/red-zone-scoring-pct?date=2022-11-10').text
#     soup = BeautifulSoup(html_text, 'lxml')
#     table = soup.find('tbody')
#     for row in table.find_all('tr'):
#         teams.append(row.find('a').text)
#         percent.append(row.find_all('td')[2].text)
#     off_red_df['Team'] = teams
#     off_red_df['off_red_per'] = percent
# get_off_red_data()

In [306]:
# # Sort teams alphabetically for easy merging
# off_red_df_sort = off_red_df.sort_values(by = 'Team').reset_index(drop=True)

In [307]:
# Create new team data df with offense redzone data
# new_teams_df = pd.concat([teams_df_sort], axis=1)
# new_teams_df['off_red_per'] = new_teams_df['off_red_per'].str.rstrip('%').astype('float') / 100.0

In [308]:
# Initialize Schedule df for 2022
# https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data?select=spreadspoke_scores.csv
grouped_schedule_df = pd.read_csv('spreadspoke_scores.csv').groupby('schedule_season')
schedule_df = pd.concat([grouped_schedule_df.get_group(group) for group in range(2019, 2023)])

In [309]:
schedule_df_2022 = schedule_df.groupby('schedule_season').get_group(2022)
schedule_df_2021 = schedule_df.groupby('schedule_season').get_group(2021)
schedule_df_2020 = schedule_df.groupby('schedule_season').get_group(2020)
schedule_df_2019 = schedule_df.groupby('schedule_season').get_group(2019)

teams_df_2022 = teams_df.groupby('Season').get_group(2022)
teams_df_2021 = teams_df.groupby('Season').get_group(2021)
teams_df_2020 = teams_df.groupby('Season').get_group(2020)
teams_df_2019 = teams_df.groupby('Season').get_group(2019)

schedule_df_2022

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
13173,9/8/2022,2022,1,False,Los Angeles Rams,10.0,31.0,Buffalo Bills,BUF,-2.0,52,SoFi Stadium,False,72.0,0.0,NaN,DOME
13174,9/11/2022,2022,1,False,Arizona Cardinals,21.0,44.0,Kansas City Chiefs,KC,-6.0,54,State Farm Stadium,False,NaN,NaN,NaN,NaN
13175,9/11/2022,2022,1,False,Atlanta Falcons,26.0,27.0,New Orleans Saints,NO,-5.5,44,Mercedes-Benz Stadium,False,NaN,NaN,NaN,NaN
13176,9/11/2022,2022,1,False,Carolina Panthers,24.0,26.0,Cleveland Browns,CAR,-1.0,42,Bank of America Stadium,False,NaN,NaN,NaN,NaN
13177,9/11/2022,2022,1,False,Chicago Bears,19.0,10.0,San Francisco 49ers,SF,-6.5,38,Soldier Field,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13475,1/1/2023,2022,100,True,Los Angeles Chargers,NaN,NaN,New York Giants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13476,1/1/2023,2022,100,True,New England Patriots,NaN,NaN,Philadelphia Eagles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13477,1/1/2023,2022,100,True,Pittsburgh Steelers,NaN,NaN,New York Jets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13478,1/1/2023,2022,100,True,Pittsburgh Steelers,NaN,NaN,Tampa Bay Buccaneers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [310]:
# Merge team data with schedule data
df_2022 = schedule_df_2022.join(teams_df_2022.set_index('Team'), on='team_home').join(teams_df_2022.set_index('Team'), on="team_away", rsuffix='_away')
df_2021 = schedule_df_2021.join(teams_df_2021.set_index('Team'), on='team_home').join(teams_df_2021.set_index('Team'), on="team_away", rsuffix='_away')
df_2020 = schedule_df_2020.join(teams_df_2020.set_index('Team'), on='team_home').join(teams_df_2020.set_index('Team'), on="team_away", rsuffix='_away')
df_2019 = schedule_df_2019.join(teams_df_2019.set_index('Team'), on='team_home').join(teams_df_2019.set_index('Team'), on="team_away", rsuffix='_away')

In [311]:
# Combine data from 2022, 2021, 2020, 2019 seasons
df = pd.concat([df_2022, df_2021, df_2020, df_2019])
# Remove playoff games
# df = df[df.schedule_playoff != True]

In [312]:
# Visualize data
df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,T_away,W-L%_away,PF_away,PA_away,PD_away,MoV_away,SoS_away,SRS_away,OSRS_away,DSRS_away
13173,9/8/2022,2022,1,False,Los Angeles Rams,10.0,31.0,Buffalo Bills,BUF,-2.0,...,0,.813,455,286,169,10.6,0.4,10.9,7.1,3.8
13174,9/11/2022,2022,1,False,Arizona Cardinals,21.0,44.0,Kansas City Chiefs,KC,-6.0,...,0,.824,496,369,127,7.5,-1.2,6.2,6.8,-0.6
13175,9/11/2022,2022,1,False,Atlanta Falcons,26.0,27.0,New Orleans Saints,NO,-5.5,...,0,.412,330,345,-15,-0.9,-0.3,-1.2,-2.8,1.7
13176,9/11/2022,2022,1,False,Carolina Panthers,24.0,26.0,Cleveland Browns,CAR,-1.0,...,0,.412,361,381,-20,-1.2,1.1,-0.1,0.7,-0.9
13177,9/11/2022,2022,1,False,Chicago Bears,19.0,10.0,San Francisco 49ers,SF,-6.5,...,0,.765,450,277,173,10.2,-2.3,7.9,3.3,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12640,12/29/2019,2019,17,False,Minnesota Vikings,19.0,21.0,Chicago Bears,CHI,-4.5,...,0,.500,280,298,-18,-1.1,0.2,-0.9,-5.4,4.5
12641,12/29/2019,2019,17,False,New England Patriots,24.0,27.0,Miami Dolphins,NE,-17.0,...,0,.313,306,494,-188,-11.8,0.2,-11.6,-2.4,-9.1
12642,12/29/2019,2019,17,False,New York Giants,17.0,34.0,Philadelphia Eagles,PHI,-3.5,...,0,.563,385,354,31,1.9,-1.7,0.3,0.7,-0.4
12643,12/29/2019,2019,17,False,Seattle Seahawks,21.0,26.0,San Francisco 49ers,SF,-3.5,...,0,.813,479,310,169,10.6,0.4,11.0,6.7,4.3


In [313]:
# Remove unnecessary columns
df.drop(schedule_df.iloc[0:0, 0:2] + schedule_df.iloc[0:0, 8:], axis=1, inplace=True)

In [314]:
df.replace('Oakland Raiders', 'Las Vegas Raiders', inplace=True)
df.replace('San Diego Chargers', 'Los Angeles Chargers', inplace=True)
df.replace('Washington Redskins', 'Washington Commanders', inplace=True)
df.replace('Washington Football Team', 'Washington Commanders', inplace=True)

In [315]:
with open(f'data.csv', 'w') as d:
    d.write(df.to_csv(index=False))